# 영화 상세 페이지 리뷰 데이터 크롤링

In [1]:
import requests
from urllib import parse
from bs4 import BeautifulSoup

import random
import time

import pandas as pd

# HTML 코드 확인

In [151]:
test_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=104426&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1"
resp = requests.get(test_url)
html = BeautifulSoup(resp.content, 'html.parser')
html


<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>네이버 영화</title>
<link href="https://ssl.pstatic.net/static/m/movie/icons/naver_movie_favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/css/common.css?20211015113922" rel="stylesheet" type="text/css">
<link href="/css/movie_tablet.css?20211015113922" rel="stylesheet" type="text/css"/>
<link href="/css/movie_end.css?20211015113922" rel="stylesheet" type="text/css"/>
<script src="/js/deploy/movie.all.js?20211015113922" type="text/javascript"></script>
</link></head>
<body>
<!-- content -->
<input id="movieCode" name="movieCode" type="hidden" value="104426"/>
<input id="onlyActualPointYn" name="onlyActualPointYn" type="hidden" value="N"/>
<input id="includeSpoilerYn" name="includeSpoilerYn" type="hidden" value="N"/>
<input id="order" name="order" type="hidden" value="sympathyScore"/>
<input id="page" name="page" type="hidden" value="1"/>
<div clas

In [62]:
score_result = html.find('div', {'class': 'score_result'})
lis = score_result.findAll('li')
lis[0]

<li>
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:100.0%"></span></span><em>10</em>
</div>
<div class="score_reple">
<p>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_0">
														
															
															
																유해진이라는 배우가 실력이 없어서 조연의 역할을 맡는게 아니라는걸 여실하게 보여준 작품. 앞으로도 주연의 위치에서 연기하는 모습을 좀 더 보고싶다. 
															
														
														
													</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(11861981, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>래기(rlaf****)</span>
</a>
</em>
<em>2016.10.13 02:33</em>
</dt>
<dd>
<a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','rlaf****', 'U6cRoccx/pwVZy1/CV+twwm5clnjm8970Zzm/mDXT04=', '유해진이라는 배우가 실력이 없어서 조연의 역할을 맡는게 아니라는걸 여실하게 보여준 작품. 앞으로도 주연의 위치에서 연기하는 모습을 좀 더 보고싶다. ', '11861981', 'point_after', false);return false;"><em>신고</em></a>
</dd>
<

In [63]:
review_text = lis[0].find('p').getText().strip()
review_text

'유해진이라는 배우가 실력이 없어서 조연의 역할을 맡는게 아니라는걸 여실하게 보여준 작품. 앞으로도 주연의 위치에서 연기하는 모습을 좀 더 보고싶다.'

In [64]:
score = lis[0].find('em').getText()
score

'10'

# 첫 번째 페이지 크롤링

In [152]:
def get_data(url):
    resp = requests.get(url)
    html = BeautifulSoup(resp.content, 'html.parser')
    score_result = html.find('div', {'class': 'score_result'})
    lis = score_result.findAll('li')
    
    for li in lis:
        score = li.find('em').getText()
        review_text = li.find('p').getText().strip()
        
        if review_text.startswith('관람객'):
            review_text = review_text[3:].strip()
        
        print(score, ':', review_text)
        print('---------------------------------------------')

In [153]:
get_data(test_url)

10 : ㅁㅊ이번에 제대로 약빨았네ㅋㅋㅋ
---------------------------------------------
9 : 자막제작자의 기지가 돋보였던 영화. 그중에서도 "백탱이" 는 매우 인상적
---------------------------------------------
10 : 1편만한 2편 요기잉네
---------------------------------------------
10 : 어째서 청불인지 궁금하다
---------------------------------------------
10 : 곰 귀여웡 하앍하앍 하야쿠
---------------------------------------------
10 : ㅋㅋㅋㅋ 역시 기대를 저버리지 않음  아만다 세레나데 장면에서 개빵터졌다 ㅋㅋ
---------------------------------------------
10 : 1편보다 미쳤엌ㅋㅋㅋㅋ 존잼 꿀잼 핵잼
---------------------------------------------
10 : ㅋㅋㅋㅋ 아 역시 테드 너무 기욥다  여전히 잔망스러워 ㅋㅋㅋ
---------------------------------------------
9 : (찡그리며)내가 이 시리얼을 가져가도 되나?
---------------------------------------------
10 : 아오랜만에 영화 재밋게봣네요
---------------------------------------------


# 전체 페이지 크롤링

In [154]:
base_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=104426&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}"

In [155]:
random.uniform(0.2, 1.2)

0.7280277093828846

In [156]:
# 결과 저장 list
comment_list = []

for page in range(1, 85):
    url = base_url.format(page)
    res = requests.get(url)
    
    if res.status_code == 200:
        soup = BeautifulSoup(resp.content, 'html.parser')
        
        score_result = html.find('div', {'class': 'score_result'})
        lis = score_result.findAll('li')
        
        for li in lis:
            score = li.find('em').getText()
            comment = li.find('p').getText().strip()
            
            # 관람객 리뷰인 경우 리뷰 앞에 '관람객' 태그가 붙으므로 제거해준다
            if comment.startswith('관람객'):
                comment = comment[3:].strip()
            
            comment_list.append((url, score, comment))
            
        # 크롤링 시 너무 자주 접근하면 사이트가 접근 방지할 수 있으므로 딜레이 걸어줌
        interval = round(random.uniform(0.2, 1.2), 2)
        time.sleep(interval)
        
print("finish")

finish


In [157]:
df = pd.DataFrame(comment_list, columns=['link', 'score', 'review'])
df.head()

,link,score,review
0,https://movie.naver.com/movie/bi/mi/pointWrite...,10,ㅁㅊ이번에 제대로 약빨았네ㅋㅋㅋ
1,https://movie.naver.com/movie/bi/mi/pointWrite...,9,"자막제작자의 기지가 돋보였던 영화. 그중에서도 ""백탱이"" 는 매우 인상적"
2,https://movie.naver.com/movie/bi/mi/pointWrite...,10,1편만한 2편 요기잉네
3,https://movie.naver.com/movie/bi/mi/pointWrite...,10,어째서 청불인지 궁금하다
4,https://movie.naver.com/movie/bi/mi/pointWrite...,10,곰 귀여웡 하앍하앍 하야쿠


In [158]:
# title의 괄호 부분에 영화 이름을 적어넣는다.
# 해당 크롤링 방식은 영화 이름을 크롤링해올 수 없기 때문에 수동으로 채워 넣어야 한다.
title = pd.DataFrame(['19곰 테드 2']*len(df), columns=['title'])
df = title.join(df)
df.head()

,title,link,score,review
0,19곰 테드 2,https://movie.naver.com/movie/bi/mi/pointWrite...,10,ㅁㅊ이번에 제대로 약빨았네ㅋㅋㅋ
1,19곰 테드 2,https://movie.naver.com/movie/bi/mi/pointWrite...,9,"자막제작자의 기지가 돋보였던 영화. 그중에서도 ""백탱이"" 는 매우 인상적"
2,19곰 테드 2,https://movie.naver.com/movie/bi/mi/pointWrite...,10,1편만한 2편 요기잉네
3,19곰 테드 2,https://movie.naver.com/movie/bi/mi/pointWrite...,10,어째서 청불인지 궁금하다
4,19곰 테드 2,https://movie.naver.com/movie/bi/mi/pointWrite...,10,곰 귀여웡 하앍하앍 하야쿠


In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   840 non-null    object
 1   link    840 non-null    object
 2   score   840 non-null    object
 3   review  840 non-null    object
dtypes: object(4)
memory usage: 26.4+ KB


In [160]:
df.to_csv('naver_movie_review_1106_Ted2.csv', encoding='euckr', index=False)